# Reservoirs
This code was used to identify reservoirs and that layer was then used to create the subregions and edges inputs for the 2022 SE blueprint

Known issues: 
1. Some barriers are not included in the National Inventory of Dams (NID) and the adjacent impounded waterbodies are therefore omitted. These are mostly farm ponds, but may also include larger impoundments.  These need to be manually added in.  
2. Some misclassifications exist in the NHD (e.g. reservoir classified as swamp/marsh or stream/river).  These need to be manually added in. 

Create a reservoir layer
Options examined for identifying “Natural” Open Water vs. "Reservoirs":

1. Pre-release of LAGOS - https://aslopubs.onlinelibrary.wiley.com/doi/10.1002/lol2.10203; Natural Lake / Reservoir Classification through machine learning; shoreline development index; NID; NWI; NHD – seems to rely most heavily on SDI a. Lake extent delineation is not robust b. I requested a pre-pub version of the data for LA and NC. NL/RES classification is ok for NC, but fails badly for LA. Examples: i. Aquaculture ponds classed as natural lakes ii. Natural coastal lakes - classed as res iii. Oxbows - classed as res
2. USGS 1938 land cover raster – a. includes oldest dams (pre-1938) b. coarse resolution (250m)
3. TNC resilient sites a. Removes all open water
4. EPA ICLUS - https://www.epa.gov/gcx/iclus-downloads a. Relies on NHD identifications of reservoir/lake/pond which is horrible.
5. Inundation Frequency (IF) is robust for open water extent at 20-30 m resolution: a. Can run for more recent years b. intersect with NLCD to exclude swamp, aquaculture
6. Shoreline Development Index (SDI) only applied to NHD-HR a. same misclassification issues as in approach #1 above
7. NHD-HR waterbodies and a. Within 200m of NID (2019) locations (https://nid.sec.usace.army.mil/ords/f?p=105:19:4229959037931::NO:::) or b. Within 50m of flowline that is within 50m of NID location

This code uses option #7. After that automated approach was applied, IF was used to guide hand editing for some of the most glaring omissions - usually caused by missing or misplacement of an NID barrier.
Note that NHD HR is huge and exceedingly difficult to manipulate due to its size.  
Preprocessing steps to select and clip down to the SE should only be run if you have a LOT of time!  

In [1]:
import os
import arcpy

In [2]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
sr_geo= arcpy.SpatialReference(4326)
#SourceWorkspace= 
OutWorkspace = r"E:\projects\BP\indicator review\Default.gdb"

In [3]:
# define rasters used for cell size, extent, and snapping
SEExtent= r"E:\projects\BP\1_Extent\BaseBPExtent2022.tif"

In [4]:
# define inputs
nhdwb= r"E:\generalGIS\NHD\NHD_H_National_GDB\NHD_H_National_GDB.gdb\Hydrography\NHDWaterbody"
nhdflowlines= r"E:\generalGIS\NHD\NHD_H_National_GDB\NHD_H_National_GDB.gdb\Hydrography\NHDFlowline"

### Start analysis

In [4]:
# Set the output workspace
arcpy.env.workspace = OutWorkspace

In [6]:
print(arcpy.env.workspace)

E:\projects\BP\indicator review\Default.gdb


In [5]:
## takes a long time - select only lake/pond and reservoir from NHD waterbodies
NHDlakepond = arcpy.management.SelectLayerByAttribute(nhdwb, "NEW_SELECTION", "FTYPE = 390 Or FTYPE = 436", None)

In [6]:
## takes a long time - select only lake/pond and reservoir in the BaseBP extent from all lakepond
NHDlakepondSE = arcpy.management.SelectLayerByLocation(NHDlakepond, "INTERSECT", "BaseBPExtent_2022", None, "SUBSET_SELECTION", "NOT_INVERT")

In [7]:
## takes a long time - save selection - 8 min
arcpy.conversion.FeatureClassToFeatureClass(NHDlakepondSE, OutWorkspace, "lakepondSE", '', 'PERMANENT_IDENTIFIER "PERMANENT_IDENTIFIER" true true false 40 Text 0 0,First,#,NHDWaterbody,PERMANENT_IDENTIFIER,0,40;FDATE "FDATE" true true false 8 Date 0 0,First,#,NHDWaterbody,FDATE,-1,-1;RESOLUTION "Resolution" true true false 4 Long 0 0,First,#,NHDWaterbody,RESOLUTION,-1,-1;GNIS_ID "GNIS_ID" true true false 10 Text 0 0,First,#,NHDWaterbody,GNIS_ID,0,10;GNIS_NAME "GNIS_NAME" true true false 65 Text 0 0,First,#,NHDWaterbody,GNIS_NAME,0,65;AREASQKM "AREASQKM" true true false 8 Double 0 0,First,#,NHDWaterbody,AREASQKM,-1,-1;ELEVATION "Elevation" true true false 8 Double 0 0,First,#,NHDWaterbody,ELEVATION,-1,-1;REACHCODE "REACHCODE" true true false 14 Text 0 0,First,#,NHDWaterbody,REACHCODE,0,14;FTYPE "FType" true true false 4 Long 0 0,First,#,NHDWaterbody,FTYPE,-1,-1;FCODE "FCode" true true false 4 Long 0 0,First,#,NHDWaterbody,FCODE,-1,-1;GLOBALID "GLOBALID" false false true 38 GlobalID 0 0,First,#,NHDWaterbody,GLOBALID,-1,-1;VISIBILITYFILTER "VisibilityFilter" true true false 4 Long 0 0,First,#,NHDWaterbody,VISIBILITYFILTER,-1,-1;SHAPE_Length "SHAPE_Length" false true true 8 Double 0 0,First,#,NHDWaterbody,SHAPE_Length,-1,-1;SHAPE_Area "SHAPE_Area" false true true 8 Double 0 0,First,#,NHDWaterbody,SHAPE_Area,-1,-1', '')

<Result 'E:\\projects\\BP\\indicator review\\Default.gdb\\lakepondSE'>

In [8]:
# takes a looooong time - select out NHD flowlines in the SE
SEFlowlines = arcpy.management.SelectLayerByLocation(nhdflowlines, "INTERSECT", "BaseBPExtent_2022", None, "NEW_SELECTION", "NOT_INVERT")
## over 2 hours and still didn't finish saving!

In [9]:
# make a layer from NID data
NID = arcpy.MakeXYEventLayer_management(r'E:\generalgis\ACE\NID2019_U.csv',"LONGITUDE","LATITUDE","NID",sr_geo)

In [10]:
# select the flowlines that are within 50m of NID locations 
#with arcpy.EnvManager(scratchWorkspace=r"E:\projects\BP\indicator review\Default.gdb", workspace=r"E:\projects\BP\indicator review\Default.gdb"):
NIDflowlines = arcpy.management.SelectLayerByLocation(SEFlowlines, "WITHIN_A_DISTANCE", NID , "50 Meters", "NEW_SELECTION", "NOT_INVERT")

In [16]:
# make a copy of the flowline selection 
flowlines_50m = arcpy.management.CopyFeatures(NIDflowlines, "flowlines_within50m_NID", '', None, None, None)

In [17]:
# select waterbodies that are close to NHDflowlines (50m) that are close to points (50m) (FROM ABOVE)
#with arcpy.EnvManager(scratchWorkspace=r"E:\projects\BP\indicator review\Default.gdb", workspace=r"E:\projects\BP\indicator review\Default.gdb"):
arcpy.management.SelectLayerByLocation(nhdSEr"E:\gflhj\Default.gdb\nhdSE", "WITHIN_A_DISTANCE", flowlines_50m, "50 Meters", "NEW_SELECTION", "NOT_INVERT")

id,value
0,a Layer object
1,nhdSE_Layer4
2,38654


In [19]:
# add to the above selection: waterbodies that are within 200m of an NID location 
#with arcpy.EnvManager(scratchWorkspace=r"E:\projects\BP\indicator review\Default.gdb", workspace=r"E:\projects\BP\indicator review\Default.gdb"):
alllake = arcpy.management.SelectLayerByLocation("nhdSE", "WITHIN_A_DISTANCE", "NID", "200 Meters", "ADD_TO_SELECTION", "NOT_INVERT")

In [20]:
# add to the above selection: waterbodies that are within 5m of a selected waterbody
#with arcpy.EnvManager(scratchWorkspace=r"E:\projects\BP\indicator review\Default.gdb", workspace=r"E:\projects\BP\indicator review\Default.gdb"):
arcpy.management.SelectLayerByLocation(alllake, "WITHIN_A_DISTANCE", alllake, "5 Meters", "ADD_TO_SELECTION", "NOT_INVERT")

id,value
0,a Layer object
1,nhdSE
2,47771


In [19]:
# the above selection is the draft reservoir selection - SAVE TO OUTPUT
arcpy.conversion.FeatureClassToFeatureClass("nhdSE_Layer4", r"E:\gflhj\Default.gdb", "NHDWB_SE", '', 'PERMANENT_IDENTIFIER "PERMANENT_IDENTIFIER" true true false 40 Text 0 0,First,#,nhdSE_Layer4,PERMANENT_IDENTIFIER,0,40;FDATE "FDATE" true true false 8 Date 0 0,First,#,nhdSE_Layer4,FDATE,-1,-1;RESOLUTION "Resolution" true true false 4 Long 0 0,First,#,nhdSE_Layer4,RESOLUTION,-1,-1;GNIS_ID "GNIS_ID" true true false 10 Text 0 0,First,#,nhdSE_Layer4,GNIS_ID,0,10;GNIS_NAME "GNIS_NAME" true true false 65 Text 0 0,First,#,nhdSE_Layer4,GNIS_NAME,0,65;AREASQKM "AREASQKM" true true false 8 Double 0 0,First,#,nhdSE_Layer4,AREASQKM,-1,-1;ELEVATION "Elevation" true true false 8 Double 0 0,First,#,nhdSE_Layer4,ELEVATION,-1,-1;REACHCODE "REACHCODE" true true false 14 Text 0 0,First,#,nhdSE_Layer4,REACHCODE,0,14;FTYPE "FType" true true false 4 Long 0 0,First,#,nhdSE_Layer4,FTYPE,-1,-1;FCODE "FCode" true true false 4 Long 0 0,First,#,nhdSE_Layer4,FCODE,-1,-1;GLOBALID "GLOBALID" false false false 38 GlobalID 0 0,First,#,nhdSE_Layer4,GLOBALID,-1,-1;VISIBILITYFILTER "VisibilityFilter" true true false 4 Long 0 0,First,#,nhdSE_Layer4,VISIBILITYFILTER,-1,-1;Shape_Length "Shape_Length" false true true 8 Double 0 0,First,#,nhdSE_Layer4,Shape_Length,-1,-1;Shape_Area "Shape_Area" false true true 8 Double 0 0,First,#,nhdSE_Layer4,Shape_Area,-1,-1', '')

<Result 'E:\\projects\\BP\\indicator review\\Default.gdb\\draft_reservoirs'>

In [8]:
# add and calculate field to use to create the raster
arcpy.management.CalculateField("draft_reservoirs", "raster", 1, "PYTHON3", '', "SHORT")

<Result 'draft_reservoirs'>

In [9]:
# convert to raster 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SAraster, snapRaster=SAraster, cellSize=SEExtent):
    arcpy.conversion.PolygonToRaster("draft_reservoirs", "raster", "res", "CELL_CENTER", "NONE", SEExtent)

In [19]:
# reclass so that reservoirs are NODATA and everyting else is 1
out_raster = arcpy.sa.IsNull("res"); out_raster.save("resNull")